In [24]:
import pandas as pd
import sqlite3 as sql
import pandas as pd
from geopy.geocoders import ArcGIS
import time
import shapely.geometry as geom
import geopandas as gpd
from __future__ import division

In [2]:
data = pd.read_csv("C:\Users\Graham\Documents\STA 141C\project\police_department_incidents.csv")

In [3]:
data['index_right'] = range(len(data))

In [4]:
# LOAD IN ZIP CODES
zipshapes = gpd.read_file("C:\Users\Graham\Documents\STA 141C\project\cb_2015_us_zcta510_500k/cb_2015_us_zcta510_500k.shp")
zipshapes.head()

# NARROW IT DOWN SOME
zipshapes = zipshapes[zipshapes.ZCTA5CE10.str.startswith("9")]

#CUT OUT UNNESSESARY STUFF
zipshapes = zipshapes[['geometry', 'ZCTA5CE10']]

In [5]:
points = gpd.GeoDataFrame([geom.asPoint(tuple(row)) for row in data[["X","Y"]].values], columns=["geometry"])

In [6]:
zipcodes = gpd.sjoin(zipshapes, points, how="inner", op="contains")

In [7]:
newdata = data.merge(zipcodes, on=['index_right'])

In [8]:
newdata['zipcode'] = [str(x) for x in newdata['ZCTA5CE10']]

In [9]:
action = ['ARREST, BOOKED', 'ARREST, CITED', 'JUVENILE BOOKED', 'PROSECUTED BY OUTSIDE AGENCY', 'JUVENILE CITED', 'JUVENILE DIVERTED'
         , 'PROSECTUTED FOR LESSER OFFENSE']

In [10]:
action_taken = [x in action for x in newdata['Resolution']]
newdata['action_taken'] = action_taken

In [11]:
newdata = newdata.drop('index_right', axis=1)
newdata = newdata.drop('ZCTA5CE10', axis=1)
newdata = newdata.drop('PdId', axis=1)

Newdata is the original dataframe with zipcodes attached. note that the same crime will be listed multiple times if it falls under several different categories

In [130]:
#FIGURE OUT HOW COMMON DIFFERENT KINDS OF CRIME ARE
counts = [len(newdata[newdata['Category']==x]) for x in list(set(newdata['Category']))]
pd.DataFrame([counts, categories]).T

,0,1
0,5132,KIDNAPPING
1,20489,WEAPON LAWS
2,23837,SECONDARY CODES
3,96238,WARRANTS
4,2389,LOITERING
5,2800,EMBEZZLEMENT
6,391,"SEX OFFENSES, NON FORCIBLE"
7,52,PORNOGRAPHY/OBSCENE MAT
8,1214,SUICIDE
9,5392,DRIVING UNDER THE INFLUENCE


In [157]:
#FIND SOME WAY TO PULL OUT THE FOLLOWING CATEGORIES
sss = ['SEX OFFENCES, NON FORCIBLE', 'TREA', 'PORNOGRAPHY/OBSCENE MAT', 'GAMBLING']
t = [newdata['Category'] == x for x in sss]

In [26]:
#67% OF ALL ARRESTS END IN NO ACTION
[sum(newdata['action_taken']==False)/len(newdata)

0.67512413389743364

In [76]:
action = [(sum(newdata[newdata['zipcode']==x]['action_taken']==True)/len(newdata[newdata['zipcode']==x]))
 for x in list(set(newdata['zipcode']))]
zips = list(set(newdata['zipcode']))

In [165]:
#RATE OF POLICE ACTIONS TAKEN ACCROSS ALL CRIMES BY ZIPCODE
action_rate_zips = pd.DataFrame([action, zips]).T
action_rate_zips.columns = ['action_taken', 'zips']

In [79]:
action = [(sum(newdata[newdata['Category']==x]['action_taken']==True)/len(newdata[newdata['Category']==x]))
 for x in list(set(newdata['Category']))]
categories = list(set(newdata['Category']))

In [170]:
#ACTION RATE BY CRIME TYPE ACCROSS ALL ZIPCODES
action_rate_category = pd.DataFrame([action, categories]).T
action_rate_category.columns = ['action_taken', 'categories']

In [171]:
action_rate_category

,action_taken,categories
0,0.447973,KIDNAPPING
1,0.700278,WEAPON LAWS
2,0.363217,SECONDARY CODES
3,0.943214,WARRANTS
4,0.874843,LOITERING
5,0.188929,EMBEZZLEMENT
6,0.273657,"SEX OFFENSES, NON FORCIBLE"
7,0.403846,PORNOGRAPHY/OBSCENE MAT
8,0.029654,SUICIDE
9,0.942507,DRIVING UNDER THE INFLUENCE


In [118]:
a = [pd.DataFrame([x, categories]) for x in oh_lawrdy]
crime_info = [x.transpose() for x in a]

In [158]:
for i in range(len(crime_info)):
    crime_info[i].columns = ['action_rate', 'category']

In [164]:
#CRIME INFO GIVES ACTION RATE BY CRIME FOR EACH INDIVIDUAL ZIPCODE
crime_info

,action_rate,category
0,0.47191,KIDNAPPING
1,0.564578,WEAPON LAWS
2,0.337509,SECONDARY CODES
3,0.922287,WARRANTS
4,0.344828,LOITERING
5,0.0877193,EMBEZZLEMENT
6,0.333333,"SEX OFFENSES, NON FORCIBLE"
7,0.272727,PORNOGRAPHY/OBSCENE MAT
8,0.0181818,SUICIDE
9,0.934426,DRIVING UNDER THE INFLUENCE


In [14]:
db = sql.connect('C:\Users\Graham\Documents\STA 141C\project\sf_data.sqlite')
cursor = db.execute("SELECT * FROM sqlite_master")
zillow = pd.read_sql("SELECT * from zillow", db)